In [140]:
import pandas as pd
import cvxpy as cp
import numpy as np
from dataclasses import dataclass
import random

# Getting People

In [141]:
def get_form_responses():
    url = r"https://docs.google.com/spreadsheets/d/1KmJBO5oKwygn-2AzwX-hS1wGeQMmLwyizFqtsSZo7_w/export?format=csv"
    df = pd.read_csv(url)
    return df

resps = get_form_responses()
resps

,Timestamp,UCLA email,First and Last name,What year are you?,Have you played with UCLA spike before?,Phone number,What is your skill level,Are you interested in playing competitively or just for fun?,Do you follow ucla.spike on instagram,How did you hear about us?,Feel free to share anything relevant,What days of Tryouts do you plan on coming? (check all that apply)
0,9/21/2025 9:46:32,youngjakecubes@g.ucla.edu,Jake Young,Senior,Yes,NaN,6 - Expert: I am willing to bet $100 nobody he...,Sign me up for sectionals,Yes,RecFest,NaN,NaN
1,9/21/2025 9:47:02,aubrey@g.ucla.edu,Aubrey Lilo,Junior,Yes,NaN,6 - Expert: I am willing to bet $100 nobody he...,Sign me up for sectionals,Yes,Glassdoor,NaN,NaN
2,9/21/2025 9:48:25,shteve@ucla.edu,Steven Pot,Freshman,No,NaN,4 - Advanced: I know about competitive serving...,Just trying to play for fun,Yes,Glassdoor,NaN,NaN


In [142]:
@dataclass
class Player:
    id: int
    name: str
    email: str
    initial_elo: float
    latest_elo: float = None

@dataclass
class Game:
    # Player IDs
    p1: int
    p2: int
    p3: int
    p4: int
    result: int  # 1 if p1/p2 win, 0 if p3/p4 win

In [143]:
c_email  = "UCLA email"
c_name = "First and Last name"
c_skill = "What is your skill level"

players = []
emails = set()
curr_id = 0
for index, row in resps.iterrows():
    if row[c_email] in emails:
        print(f"Duplicate email found: {row[c_email]}, skipping entry.")
        print(row)
    skill = int(row[c_skill][0])
    initial_elo = 600 + skill * 100
    player = Player(curr_id, row[c_name], row[c_email], initial_elo)
    players.append(player)
    emails.add(row[c_email])
    curr_id += 1
    print(f"Added player: {player.name} with ID: {player.id}")

Added player: Jake Young with ID: 0
Added player: Aubrey Lilo with ID: 1
Added player: Steven Pot with ID: 2


In [83]:
print(players[0])

Player(id=0, name='Jake Young', email='youngjakecubes@g.ucla.edu', initial_elo=1200, latest_elo=None)


# Getting Games

## todo lol

In [84]:
def get_games():
    url = r"https://docs.google.com/spreadsheets/d/1JGXGaMiixEOw_vLqbQ9XwBn8BoNYGGN9PEqI7RWOtKg/export?format=csv"
    df = pd.read_csv(url)
    return df

games = get_games()

In [85]:
games

,Timestamp,Player 1 id,Player 2 id,Player 3 id,Player 4 id,Result
0,9/21/2025 10:43:47,0,1,2,3,1
1,9/21/2025 10:43:59,0,2,1,3,1


# Optimizing Ratings

In [144]:
def optimize_ratings(players, games):
    for index, player in enumerate(players):
        assert player.id == index, "Player IDs must be sequential starting from 0"
    
    beta = np.log(10) / 400
    n = len(players)
    initial_ratings = np.array([player.initial_elo for player in players]) # modify this according to input skill levels
    sigma = 400.0
    ratings = cp.Variable(n)
    # prior is rating is normal dist with stdev 400, mean self reported
    self_report_prior = -0.5 * cp.sum_squares((ratings - initial_ratings) / sigma)
    # probably set to 1000
    mean_prior = 10 * -0.5 * cp.sum_squares((cp.sum(ratings) / n - 1500) / (sigma / np.sqrt(n)))

    obj_func = []
    for g in games:
        result = float(g.result) # 1 if p1/p2 win, 0 if p3/p4 win
        z = beta * 0.5 * (ratings[g.p1] + ratings[g.p2] - ratings[g.p3] - ratings[g.p4])
        obj_func.append(result * z - cp.logistic(z))

    objective = cp.Maximize(cp.sum(obj_func) + self_report_prior + mean_prior)
    constraints = [ratings >= 0, ratings <= 2000]
    problem = cp.Problem(objective, constraints)
    problem.solve()
    if True:
        for i in range(n):
            players[i].latest_elo = int(ratings.value[i])
    return ratings.value

# Testing Optimizer

In [149]:
# Josh: 1900 Kyan: 1750 Jake: 1650 Jai: 1625 Archie: 1600 Steven: 1550 Reece: 1450 Luke: 1350 Aubrey: 1250 Alex: 1000
true_elos = {
    "Josh": 1900,
    "Kyan": 1750,
    "Jake": 1650,
    "Jai": 1625,
    "Archie": 1600,
    "Steven": 1550,
    "Aubrey": 1500,
    "Reece": 1450,
    "Luke": 1350,
    "Alex": 1000
}

true_ratings = list(true_elos.values())
names = list(true_elos.keys())

players = [Player(i, list( true_elos.keys())[i], "", 1500) for i in range(10)]


In [150]:
print(players)
print(np.mean(true_ratings))

[Player(id=0, name='Josh', email='', initial_elo=1500, latest_elo=None), Player(id=1, name='Kyan', email='', initial_elo=1500, latest_elo=None), Player(id=2, name='Jake', email='', initial_elo=1500, latest_elo=None), Player(id=3, name='Jai', email='', initial_elo=1500, latest_elo=None), Player(id=4, name='Archie', email='', initial_elo=1500, latest_elo=None), Player(id=5, name='Steven', email='', initial_elo=1500, latest_elo=None), Player(id=6, name='Aubrey', email='', initial_elo=1500, latest_elo=None), Player(id=7, name='Reece', email='', initial_elo=1500, latest_elo=None), Player(id=8, name='Luke', email='', initial_elo=1500, latest_elo=None), Player(id=9, name='Alex', email='', initial_elo=1500, latest_elo=None)]
1537.5


In [151]:
rng = random.Random(100)
num_games = 1000
games = []

indices = list(range(len(players)))
for _ in range(num_games):
    # pick 4 distinct players
    picks = rng.sample(indices, 4)
    rng.shuffle(picks)
    p1, p2, p3, p4 = picks[0], picks[1], picks[2], picks[3]

    # true win prob for Team1 using Elo model
    D_true = 0.5 * (true_ratings[p1] + true_ratings[p2] - true_ratings[p3] - true_ratings[p4])
    p_win = 1.0 / (1.0 + 10 ** (-D_true / 400.0))

    # sample outcome
    result = 1 if rng.random() < p_win else 0

    games.append(Game(p1=p1, p2=p2, p3=p3, p4=p4, result=result))

# ----- Fit ratings with your optimizer -----
estimated = optimize_ratings(players, games)

# Matchmaking